In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Collection

#### Using API

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_crime_rate'

In [3]:
try:
    resp = requests.get(url)
except Exception as e:
    print("Invalid url")
    raise e

In [4]:
soup = BeautifulSoup(resp.text, 'html')
table = soup.find_all('table')[0]

## Data Preprocessing

#### Select column names

In [5]:
data_columns = table.find_all('th')
data_columns = [name.text.strip() for name in data_columns]

#### Remove columns

In [6]:
data_columns.remove('Yearly Crime Rates per 100,000 people')
data_columns.remove('Violent crime')
data_columns.remove('Property crime')

#### Rename columns

In [7]:
data_columns[3] = 'Total (Violent and Property)'
data_columns[9] = 'Total Violent'
data_columns[13] = 'Total Property'

#### Reorder columns

In [8]:
element = data_columns.pop(4)
data_columns.insert(13, element)

#### Store as DataFrame

In [9]:
crime_data = pd.DataFrame(columns = data_columns)
column_data = table.find_all('tr')

In [10]:
for row in column_data[3:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]
    crime_data.loc[len(crime_data)] = individual_row_data

#### Fix values with numbers

In [11]:
crime_data.at[0, 'City'] = 'Mobile'
crime_data.at[26, 'City'] = 'Stockton'
crime_data.at[73, 'City'] = 'Toledo'
crime_data.at[81, 'City'] = 'Arlington'
crime_data.at[47, 'City'] = 'Louisville Metro'

#### Rename columns to be clearer

In [12]:
new_columns = {'Murder andNonnegligentmanslaughter': 'Murder/Manslaughter', 
               'Rape1': 'Rape',
               'Aggravatedassault': 'Assault',
               'Larceny-theft': 'Larceny/Theft',
               'Motorvehicletheft': 'Vehicle Theft',
               'Arson2' : 'Arson'}
crime_data.rename(columns = new_columns, inplace=True)

In [13]:
#View data
crime_data

,State,City,Population,Total (Violent and Property),Murder/Manslaughter,Rape,Robbery,Assault,Total Violent,Burglary,Larceny/Theft,Vehicle Theft,Total Property,Arson
0,Alabama,Mobile,"248,431",6217.02,20.13,58.16,177.11,485.85,740.25,"1,216.84","3,730.21",506.78,"5,453.83",22.94
1,Alaska,Anchorage,"296,188",6640.04,9.12,132.01,262.67,799.49,"1,203.29",748.17,"3,619.66","1,047.98","5,415.82",20.93
2,Arizona,Chandler,"249,355",2589.08,2.01,52.13,56.95,148.68,259.47,314.41,"1,866.01",149.18,"2,329.61",
3,Arizona,Gilbert,"242,090",1483.75,2.07,16.11,21.07,46.26,85.51,192.49,"1,137.59",55.76,"1,385.85",12.39
4,Arizona,Glendale,"249,273",5037.85,4.81,38.91,192.96,251.53,488.22,637.45,"3,426.36",466.56,"4,530.37",19.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Virginia,Virginia Beach,"454,353",2083.4,3.08,22.45,59.87,52.16,137.56,165.95,"1,667.65",101.02,"1,934.62",11.22
96,Washington,Seattle,"721,365",5911.99,3.74,36.87,210.02,382.05,632.69,"1,081.98","3,673.45",503.21,"5,258.64",20.66
97,Washington,Vancouver,"217,066",7881.01,2.76,109.64,105.50,408.63,626.54,"1,023.19","5,408.03",800.22,"7,231.44",23.03
98,Wisconsin,Madison,"255,850",3040.46,4.30,37.13,82.86,250.15,374.44,362.32,"2,130.94",168.85,"2,662.11",3.91


In [14]:
#Dimensions
num_rows, num_columns = crime_data.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 100
Number of columns: 14


In [15]:
#Save as csv file
#crime_data.to_csv('crime_data.csv')

#### Check data types (?)

In [16]:
crime_data.dtypes

State                           object
City                            object
Population                      object
Total (Violent and Property)    object
Murder/Manslaughter             object
Rape                            object
Robbery                         object
Assault                         object
Total Violent                   object
Burglary                        object
Larceny/Theft                   object
Vehicle Theft                   object
Total Property                  object
Arson                           object
dtype: object

In [17]:
#Convert object to numeric
crime_data['Population'] = pd.to_numeric(crime_data['Population'].replace(',', '', regex=True), errors='coerce')
crime_data['Total Property'] = pd.to_numeric(crime_data['Total Property'].replace(',', '', regex=True), errors='coerce')
crime_data['Larceny/Theft'] = pd.to_numeric(crime_data['Larceny/Theft'].replace(',', '', regex=True), errors='coerce')
crime_data['Burglary'] = pd.to_numeric(crime_data['Burglary'].replace(',', '', regex=True), errors='coerce')
crime_data['Vehicle Theft'] = pd.to_numeric(crime_data['Vehicle Theft'].replace(',', '', regex=True), errors='coerce')
crime_data['Assault'] = pd.to_numeric(crime_data['Assault'].replace(',', '', regex=True), errors='coerce')
crime_data['Rape'] = pd.to_numeric(crime_data['Rape'].replace(',', '', regex=True), errors='coerce')
crime_data['Arson'] = pd.to_numeric(crime_data['Arson'].replace(',', '', regex=True), errors='coerce')
crime_data['Total Violent'] = pd.to_numeric(crime_data['Total Violent'].replace(',', '', regex=True), errors='coerce')

crime_data['Total (Violent and Property)'] = pd.to_numeric(crime_data['Total (Violent and Property)'], errors='coerce')
crime_data['Murder/Manslaughter'] = pd.to_numeric(crime_data['Murder/Manslaughter'], errors='coerce')
crime_data['Robbery'] = pd.to_numeric(crime_data['Robbery'], errors='coerce')

#### Check for NA

In [22]:
missing_values = crime_data.isnull().sum()
print("Missing values in DataFrame:")
print(missing_values)

Missing values in DataFrame:
State                           0
City                            0
Population                      0
Total (Violent and Property)    0
Murder/Manslaughter             0
Rape                            0
Robbery                         0
Assault                         0
Total Violent                   0
Burglary                        0
Larceny/Theft                   0
Vehicle Theft                   0
Total Property                  0
Arson                           0
dtype: int64


#### Check for Duplicates

In [16]:
duplicates_values = crime_data.duplicated().sum()
print("Missing values in DataFrame:")
print(duplicates_values)

Missing values in DataFrame:
0


#### Handle Missing Values

In [21]:
#Columns wiht missing values: Rape, Total Violent, and Arson
missingval_col = ['Rape', 'Total Violent', 'Arson']

#Get mean within each column
mean_values = crime_data[missingval_col].mean() 

#Fill with mean imputation
crime_data[missingval_col] = crime_data[missingval_col].fillna(mean_values)

#### Descriptive statistics

In [23]:
#Only looking at crimes
selected_columns = ['Murder/Manslaughter', 'Rape', 'Robbery', 'Assault', 'Burglary',  'Larceny/Theft',  'Vehicle Theft', 'Arson']

# Create a new DataFrame with the selected columns
crime_only = crime_data[selected_columns].copy()

In [24]:
statistics = crime_only.describe().sort_values(by='mean', axis=1, ascending = False)
statistics 

,Larceny/Theft,Burglary,Vehicle Theft,Assault,Robbery,Rape,Arson,Murder/Manslaughter
count,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000
mean,2466.824000,647.612300,477.689900,430.54330,229.274100,59.942083,24.661957,11.618400
std,953.287709,323.872754,283.276318,287.05522,159.584269,28.229882,20.009538,10.940627
min,915.500000,128.870000,55.760000,23.90000,19.920000,13.850000,0.730000,0.720000
25%,1813.807500,382.092500,285.805000,249.33250,127.540000,39.165000,10.640000,4.185000
50%,2378.305000,599.735000,451.390000,399.59000,191.090000,58.290000,19.995000,8.595000
75%,2921.770000,833.040000,592.072500,530.66500,306.512500,73.595000,29.760000,15.702500
max,5408.030000,1518.870000,1368.780000,1519.55000,958.710000,144.670000,129.550000,66.070000


* Larceny/Theft has the highest average while Murder/Manslaughter has the lowest average

## Data Visualizations